In [73]:
import pandas as pd;


Start_date="1-Jan-2019"
Start_date=input("Enter Start date in format.  1-Jan-2019 ")
End_date="31-Jan-2019"
End_date=input("Enter END date.  in format.  1-Jan-2019")
calldf=pd.read_csv("OPTIDX_NIFTY_CE_"+Start_date+"_TO_"+End_date+".csv",index_col=['Strike Price','Date'])




# calldf=pd.read_csv("OPTIDX_NIFTY_CE_1-Jan-2019_TO_31-Jan-2019.csv",index_col=['Strike Price','Date'])
calldf=calldf.drop(['Symbol','Open','High', 'Low', 'Close', 'LTP','No. of contracts','Turnover in Lacs', 'Premium Turnover in Lacs', 'Open Int',
       'Change in OI'],axis=1)
date=calldf.index.get_level_values(0).unique().tolist()
list(date)
strike=calldf.index.unique()
list(strike)
settle=calldf['Settle Price']
list(settle)
strikep=calldf.index.get_level_values(0).unique().tolist()
date=calldf.index.get_level_values(1).unique().tolist()




df=pd.read_csv("OPTIDX_NIFTY_PE_"+Start_date+"_TO_"+End_date+".csv",index_col=['Strike Price','Date'])
df=df.drop(['Symbol','Open','High', 'Low', 'Close', 'LTP','No. of contracts','Turnover in Lacs', 'Premium Turnover in Lacs', 'Open Int',
       'Change in OI'],axis=1)
k=pd.DataFrame(data=df['Settle Price'],columns=list(date),index=list(strikep))
call=pd.DataFrame(data=calldf['Settle Price'],columns=list(date),index=list(strikep))
for i in strikep:
    for j in date:
        if (i,j) in df.index:
            l=df.loc[[(i,j)],'Settle Price']
            k.at[i,j]=l[0]



for i in strikep:
    for j in date:
        if (i,j) in calldf.index:
            l=df.loc[[(i,j)],'Settle Price']
            k.at[i,j]=l[0]

            l=calldf.loc[[(i,j)],'Settle Price']
            call.at[i,j]=l[0]
#call variable has callDATA

# date=df.index.get_level_values(0).unique().tolist()
# list(date)
# strike=df.index.unique()
# list(strike)
# settle=df['Settle Price']
# list(strike)
# strikep=df.index.get_level_values(0).unique().tolist()
# date=df.index.get_level_values(1).unique().tolist()

#k variable has put data



In [74]:
#--------------------------------------------  PRICES CONIDERED.   ---------------------------------#
ATM=int(input("ENTER SPOT PRICE"))
CALLOTM=ATM+500
PUTOTM=ATM-500

#---------------------------------------------- Concat finalData from both sheets---------------------#
finaldata=pd.concat([call,k],keys=['call', 'put'])

#----------------------------------selecting specific Strike price from Data. -------------------------------#
for i in strikep:
    if ATM==i:
        finalfilter=finaldata.loc[(['call','put'],[ATM,CALLOTM,PUTOTM]),:]
finalfilter







#list used for storing profit data 
prof_calotm=[]  #profit from buying call otm
prof_calatm=[]  #profit from selling call otm
prof=[]         #Total profit from CALLS

prof_putotm=[]
prof_putatm=[]
profP=[]

finalProfit=[]

emptyrow=[]
#for getting spread data of both sell 
Profitbothsell=[] # add on selling both sides sell spread
callitmsellprof=[]
putitmsellprof=[]

atmprice_day1=finalfilter.loc[('call',ATM),finalfilter.columns[0]]
otmprice_day1=finalfilter.loc[('call',CALLOTM),finalfilter.columns[0]]

atmprice_day1P=finalfilter.loc[('put',ATM),finalfilter.columns[0]]
otmprice_day1P=finalfilter.loc[('put',PUTOTM),finalfilter.columns[0]]


#for getting spread data of both sell 
call_itm_price_day1C=finalfilter.loc[('call',PUTOTM),finalfilter.columns[0]]
put_itm_rice_day1P=finalfilter.loc[('put',CALLOTM),finalfilter.columns[0]]





for index,i in enumerate(finalfilter.columns):

    atmprice_day_FINAL=finalfilter.loc[('call',ATM),i]
    otmprice_day_FINAL=finalfilter.loc[('call',CALLOTM),i]

    atmprice_day_FINALP=finalfilter.loc[('put',ATM),i]
    otmprice_day_FINALP=finalfilter.loc[('put',PUTOTM),i]


    prof_calotm.append(otmprice_day_FINAL-otmprice_day1)
    prof_calatm.append(atmprice_day1-atmprice_day_FINAL)

    prof_putotm.append(otmprice_day_FINALP-otmprice_day1P)
    prof_putatm.append(atmprice_day1P-atmprice_day_FINALP)
    emptyrow.append(" ")






    prof.append(prof_calotm[index]+prof_calatm[index])

    profP.append(prof_putotm[index]+prof_putatm[index])

    finalProfit.append(prof[index]+profP[index])

    #for getting spread data of both sell 
    call_itm_price_day_final=finalfilter.loc[('call',PUTOTM),i]
    put_itm_rice_day_final=finalfilter.loc[('put',CALLOTM),i]
    callitmsellprof.append(call_itm_price_day1C-call_itm_price_day_final)
    putitmsellprof.append(put_itm_rice_day1P-put_itm_rice_day_final)




    Profitbothsell.append(callitmsellprof[index]+putitmsellprof[index])








profitsdf=pd.DataFrame(list(zip(prof_calotm,prof_calatm,prof,prof_putotm,prof_putatm,profP,finalProfit,emptyrow,callitmsellprof,putitmsellprof,Profitbothsell)),columns=['CALL OTM PROFITS','CALL ATM PROFITS','TOTAL CALL PROFITS','PUT OTM PROFITS','PUT ATM PROFITS','TOTAL PUT PROFITS','Final Profits','EMPTY ROW','SELL CALL ITM PROF','SELL PUT ITM PROF','BOTHSELL RANGE ITM'],index=finalfilter.columns)
profitsdft=profitsdf.T
profitsdft['PRICE AT FIRT DATE']=list(profitsdft[profitsdft.columns[0]])
profitsdft=profitsdft.set_index('PRICE AT FIRT DATE',append=True)
profitsdft








finalsheet=pd.concat([finalfilter,profitsdft],keys=['Data','Profits'])
from datetime import datetime
dt=datetime.now().strftime("%y_%m_%d:%I_%M_%S")
dt
finalsheet.to_excel("ANALYSE"+dt+".xlsx")


# Depiction added


import openpyxl

from openpyxl.styles import PatternFill
from openpyxl.styles.borders import Border, Side
from openpyxl.workbook import Workbook
from openpyxl import load_workbook


workbook = load_workbook(filename="ANALYSE"+dt+".xlsx")
ws=workbook['Sheet1']
ws
fillpat_green=PatternFill(patternType='darkGray',fgColor='079C6F')
fillpat_red=PatternFill(patternType='darkGray',fgColor='F36161')
thin_border = Border(left=Side(style='thin'), 
                     right=Side(style='thin'), 
                     top=Side(style='thin'), 
                     bottom=Side(style='thin'))
ws.column_dimensions['B'].width=30
for row_cells in ws.iter_rows(min_row=8):
    for cell in row_cells:
        if type(cell.value)==int or type(cell.value)==float:
            if cell.value>=0:
                cell.fill=fillpat_green
                cell.border=thin_border
            else :
                cell.fill=fillpat_red
                cell.border=thin_border
       #print('%s: cell.value=%s' % (cell, cell.value) )

# if ws['D4'].value>=0:
#     ws['D4'].fill=fillpat_green
workbook.save("ANALYSE"+dt+".xlsx")



01-Jan-2019 02-Jan-2019 03-Jan-2019 04-Jan-2019 07-Jan-2019  \
call 10300       684.2      569.95      473.05       516.9      528.35   
     10800       276.7       204.5       145.1       163.1       167.5   
     11300       42.75        28.4        17.1        17.3        15.2   
put  10300        30.9        45.1       62.35        46.1       40.15   
     10800       117.8      172.05       224.5       184.6       165.9   
     11300         374      489.55      588.45         530      502.75   

           08-Jan-2019 09-Jan-2019 10-Jan-2019 11-Jan-2019 14-Jan-2019  ...  \
call 10300      563.45      587.75      565.45      535.35      486.25  ...   
     10800       182.2       193.6      179.25       156.8       121.5  ...   
     11300        14.8       13.95       11.75         8.8         5.6  ...   
put  10300        29.8        23.7       22.15       21.65        26.3  ...   
     10800       138.9       118.4       122.3      130.25      153.95  ...   
     11300      469.25      434.75      451.05      482.05      534.75  ...   

           18-Jan-2019 21-Jan-2019 22-Jan-2019 23-Jan-2019 24-Jan-2019  \
call 10300      630.15      669.35      640.25       566.2      575.15   
     10800       190.1      224.25      192.65      134.75      127.95   
     11300        7.95        7.65        5.55         3.4        2.25   
put  10300        9.25        6.85         5.7         5.9        3.45   
     10800       67.65       57.15       58.75       81.85        62.1   
     11300      378.15      337.25      366.15       440.9       426.8   

           25-Jan-2019 28-Jan-2019 29-Jan-2019 30-Jan-2019 31-Jan-2019  
call 10300      492.75      380.35       377.3      336.95           0  
     10800          74       29.95       17.55        3.45           0  
     11300         1.6         1.9        1.15        0.35           0  
put  10300         4.6         5.2         3.4        1.15           0  
     10800        92.1       153.6      138.65      161.55           0  
     11300      509.75      619.35      625.05       641.4           0  

[6 rows x 23 columns]

In [84]:

# CE=str(input("ENTER CE OR PE"))
Start_date="1-Jan-2019"
Start_date=input("Enter Start date in format.  1-Jan-2019 ")
End_date="31-Jan-2019"
End_date=input("Enter END date.  in format.  1-Jan-2019")
check=pd.read_csv("OPTIDX_NIFTY_CE_"+Start_date+"_TO_"+End_date+".csv",index_col=['Strike Price','Date'])
check

,,Symbol,Expiry,Option Type,Open,High,Low,Close,LTP,Settle Price,No. of contracts,Turnover in Lacs,Premium Turnover in Lacs,Open Int,Change in OI,Underlying Value
Strike Price,Date,,,,,,,,,,,,,,,
11750,01-Jan-2019,NIFTY,31-Jan-2019,CE,3.75,3.90,3.00,3.55,3.75,3.55,13,114.60,0.03,675,300,10910.1
11800,01-Jan-2019,NIFTY,31-Jan-2019,CE,2.50,3.10,2.45,3.00,2.95,3.00,3311,29309.05,6.70,247350,-6750,10910.1
11850,01-Jan-2019,NIFTY,31-Jan-2019,CE,2.50,3.35,2.05,3.35,3.35,3.35,8,71.12,0.02,825,75,10910.1
11900,01-Jan-2019,NIFTY,31-Jan-2019,CE,2.50,2.50,1.90,2.05,1.90,2.05,498,4445.40,0.75,55050,-2475,10910.1
11950,01-Jan-2019,NIFTY,31-Jan-2019,CE,0.80,2.50,0.75,2.30,2.30,2.30,6,53.78,0.01,1950,225,10910.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11500,31-Jan-2019,NIFTY,31-Jan-2019,CE,0.50,0.50,0.05,0.05,0.05,0.00,6665,57485.88,0.26,1609350,49125,10830.95
11550,31-Jan-2019,NIFTY,31-Jan-2019,CE,0.40,0.40,0.05,0.05,0.05,0.00,49,424.47,0.01,9300,1575,10830.95
11600,31-Jan-2019,NIFTY,31-Jan-2019,CE,0.10,0.10,0.05,0.05,0.05,0.00,215,1870.51,0.01,434775,-10425,10830.95


[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ' ', 0.0, 0.0, 0.0]

'21_05_23:10_01_48'